In [22]:

import json # Importamos json para leer nuestros json
import ast
import pandas as pd  # Importamos pandas para trabajar con nuestros datasets
import re # Importamos re para modificar nuestros strings


In [23]:

df_games_steam = pd.read_json('../Data/australian_output_steam_games.json', lines='true')
df_games_steam.describe()



,early_access,id
count,32135.000000,3.213300e+04
mean,0.060588,4.517574e+05
std,0.238577,1.827140e+05
min,0.000000,1.000000e+01
25%,0.000000,3.292800e+05
50%,0.000000,4.520600e+05
75%,0.000000,5.934000e+05
max,1.000000,2.028850e+06


In [9]:
df_games_steam



,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [10]:

df_games_steam['price'] = df_games_steam['price'].replace('Free To Play', 0) # Convertimos los precios 'Free to play' a 0 
df_games_steam['price'] = pd.to_numeric(df_games_steam['price'], errors='coerce') # Convertimos a enteros los precios, salteando errores
df_games_steam.reset_index(inplace=True, drop=True) # Reseteamos el indice


In [11]:
df_games_steam['app_name'] = df_games_steam['app_name'].str.lower() # Convertimos nuestros juegos a minusculas
df_games_steam['price'].fillna(0, inplace=True) # Los precios nulos los convertimos a 0, suponiendo que significan gratis
df_games_steam['early_access'] = df_games_steam['early_access'].dropna()
# df_games_steam['early_access'] = df_games_steam['early_access'].astype(int) # Convertimos a 1s y 0s
df_games_steam['genres'].fillna('[]', inplace=True) # Generos nulos a listas vacias
df_games_steam['release_date'].fillna(0, inplace=True)# Las fechas nulas las convertimos a 0_steam


In [12]:
df_games_steam.dropna(inplace=True) # Eliminamos valores faltantes

invalid_dates = df_games_steam["release_date"][pd.to_datetime(df_games_steam["release_date"], errors='coerce').isna()]
#Extraemos las fechas que estan en formato incorrecto



In [13]:
# Creamos una funcion que transforma a 0 las fechas que no contienen numeros
def transform_to_zero(x):
    if x == 0:
        return 0
    if not re.search(r'\d', x):
        return 0
    return x
df_games_steam['release_date'] = df_games_steam['release_date'].apply(transform_to_zero) # La aplicamos a las fechas


In [14]:
invalid_dates = df_games_steam["release_date"][pd.to_datetime(df_games_steam["release_date"], errors='coerce').isna()]
invalid_dates.unique() # Mostramos nuestras fechas invalidas


array([0, '2017', 'Jun 2009', 'Oct 2010', 'Feb 2011', 'Aug 2014',
       'Sep 2014', 'Apr 2015', 'Apr 2016', 'June 2016', 'Q2 2017',
       'Q2 2018', '2018', 'Winter 2017', 'Mar 2018', '14 July',
       'Jul 2017', 'Summer 2017', 'Spring 2018', 'Winter 2018',
       'October 2017', 'Fall 2017', 'Nov 2017', 'Q1 2018', 'Dec 2017',
       'Январь 2018', 'First quarter of 2018', 'H2 2018', '21 Jun, 2017',
       '2018年初頭発売予定', 'Feb 2018', 'August 2017',
       '2018 [Now get free Pre Alpha]', '1st Quarter 2018', 'Jan 2018',
       '预热群52756441', '2018年1月', 'Fall 2018',
       '0̵1̴0̵0̶1̷0̶0̵0̴ ̴0̶0̶1̶1̶0̷0̶1̵1̴ ̸0̶0̶1̶1̵0̶1̷0̴0̵ ̴0̶1̷0̸1̵0̷0̴1̶0̴ ̴0̷0̴1̷1̶0̶1̵1̷1̵ ̵',
       'Spring 2017', 'Coming 2017', 'January 2019', 'Q4 2017',
       'Early 2018', 'Q2 2018 (Tentative)', 'Oct 2017', '2017 Q4',
       'August 2018', 'Early Spring 2018', 'TBA 2017', 'Aug 2017',
       'Coming Q1 2018', 'December 2017', 'Apr 2017', 'Coming Q3 2017',
       'Early 2017', 'End 2017', 'Q1 2017', 'Jan 2017', 

In [15]:
# Creamos una funcion que extrae el año de las fechas en las que puede hacerlo
def extract_year(date_str):
    date_str = str(date_str)
    year_match = re.search(r'\d{4}', date_str)
    if year_match:
        return str(year_match.group())
    else:
        return None

# Aplicar la función a la columna release_date y creamos una nueva columna año con nuestros años restaurados
df_games_steam['year'] = df_games_steam['release_date'].apply(lambda x: extract_year(x))


In [16]:
df_games_steam = df_games_steam.explode('genres') # Explotam# Eliminamos release_date 
df_games_steam.drop('release_date', axis=1, inplace=True) # 


In [17]:
df_games_steam.year.unique() # Visualizamos nuestros años unicos (1970-2021)


array(['2018', '2017', None, '1997', '1998', '2016', '2006', '2005',
       '2003', '2007', '2002', '2000', '1995', '1996', '1994', '2001',
       '1993', '2004', '1999', '2008', '2009', '1992', '1989', '2010',
       '2011', '2013', '2012', '2014', '1983', '1984', '2015', '1990',
       '1988', '1991', '1987', '1986', '2021', '5275', '2019', '1985'],
      dtype=object)

In [18]:
df_games_steam.loc[df_games_steam.year == '5275', 'year'] = None # Tenemos una fecha incorrecta (5275)


In [19]:
df_games_steam.tail()


,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,year
120442,Laush Studio,Indie,russian roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,2018
120442,Laush Studio,Racing,russian roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,2018
120442,Laush Studio,Simulation,russian roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich,2018
120443,SIXNAILS,Casual,exit 2 - directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns",2017
120443,SIXNAILS,Indie,exit 2 - directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns",2017


In [20]:
df_games_steam = df_games_steam.drop(['tags', 'specs', 'url', 'reviews_url'], axis=1)
df_games_steam.columns


Index(['publisher', 'genres', 'app_name', 'title', 'price', 'early_access',
       'id', 'developer', 'year'],
      dtype='object')

In [21]:
df_games_steam.to_parquet(r'../Data/clean_games_steam.parquet.gzip', compression='gzip') # Exportamos 


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.